In [31]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy import ndarray

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import lincoln
from lincoln.numpy.layers import Dense
from lincoln.numpy.losses import MeanSquaredError
from lincoln.numpy.optimizers import SGD
from lincoln.numpy.activations import Sigmoid, Tanh, Linear
from lincoln.numpy.network import NeuralNetwork

## Regression network

In [34]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

## Regression models

#### Linear Regression

In [35]:
from lincoln.numpy.train import Trainer

In [36]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(lr, SGD(lr=0.001))

In [37]:
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 3000,
       eval_every = 300,
       seed=181008,
           restart=True);

Validation loss after 300 epochs is 26.057
Validation loss after 600 epochs is 25.836
Validation loss after 900 epochs is 25.456
Loss increased after epoch 1200, final loss was 25.456, using the model from epoch 900


In [38]:
# test "restart"
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 10,
       seed=181008,
           restart=True);

Validation loss after 10 epochs is 321.210
Validation loss after 20 epochs is 203.308
Validation loss after 30 epochs is 134.012
Validation loss after 40 epochs is 90.925
Validation loss after 50 epochs is 65.545


In [39]:
def mae(y_true, y_pred):
    
#     assert_same_shape(y_true, y_pred)
    
    return np.round(np.mean(np.abs(y_true - y_pred)), 4)

def mse(y_true, y_pred):
    
#     assert_same_shape(y_true, y_pred)
    
    return np.round(np.mean(np.power(y_true - y_pred, 2)), 4)

def rmse(y_true, y_pred):
    
#     assert_same_shape(y_true, y_pred)
    
    return np.round(np.sqrt(np.mean(np.power(y_true - y_pred, 2))), 4)

In [40]:
def eval_regression_model(model: NeuralNetwork, 
                          X_test: ndarray, 
                          y_test: ndarray):
    preds = model.forward(X_test, inference=True)
    preds = preds.reshape(-1, 1)
    print("Mean absolute error: {:.2f}".format(mae(preds, y_test)))
    print()
    print("Mean squared error {:.2f}".format(mse(preds, y_test)))
    print()
    print("Root mean squared error {:.2f}".format(rmse(preds, y_test)))

In [41]:
eval_regression_model(lr, X_test, y_test)

Mean absolute error: 3.57

Mean squared error 25.61

Root mean squared error 5.06


Works!

In [42]:
neural_network = NeuralNetwork(
    layers=[Dense(neurons=13,
                  activation=Sigmoid()),
            Dense(neurons=1,
                  activation=Linear())],
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(neural_network, SGD(lr=0.001))

trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 3000,
       eval_every = 300,
       seed=181008);

Validation loss after 300 epochs is 20.639
Validation loss after 600 epochs is 16.549
Validation loss after 900 epochs is 13.950
Validation loss after 1200 epochs is 13.425
Validation loss after 1500 epochs is 13.088
Validation loss after 1800 epochs is 12.687
Validation loss after 2100 epochs is 12.614
Validation loss after 2400 epochs is 12.536
Loss increased after epoch 2700, final loss was 12.536, using the model from epoch 2400


In [43]:
eval_regression_model(neural_network, X_test, y_test)

Mean absolute error: 2.29

Mean squared error 12.56

Root mean squared error 3.54


In [49]:
params = []
for layer in neural_network.layers:
    for op in layer.operations[:2]:
        params.append(op.param)

In [54]:
def np_sigmoid(x) -> np.ndarray:
    return 1.0/(1.0+np.exp(-1.0 * x))

In [57]:
layer1 = np.dot(X_test, params[0]) + params[1]
out1 = np_sigmoid(layer1)

array([[47.33268377],
       [ 8.98516824],
       [13.91075996],
       [ 7.93468361],
       [16.13168504],
       [20.5552775 ],
       [11.68654007],
       [19.87925746],
       [33.24143095],
       [22.97014045],
       [32.93132696],
       [17.72598242],
       [13.81478134],
       [40.66145341],
       [13.8490556 ],
       [23.08421123],
       [11.34911861],
       [30.9404698 ],
       [19.67914775],
       [19.95585143],
       [22.64909092],
       [24.26664269],
       [11.36156652],
       [22.11381357],
       [16.3174684 ],
       [24.00972524],
       [22.98795928],
       [36.21887023],
       [31.42414979],
       [19.91930403],
       [ 7.04070053],
       [ 6.40581856],
       [27.3933198 ],
       [19.84623511],
       [18.39812903],
       [44.2827755 ],
       [39.27353192],
       [21.82063106],
       [23.72349305],
       [25.77403619],
       [37.45999248],
       [19.53796451],
       [41.60743969],
       [15.61976654],
       [28.80659748],
       [26

In [60]:
out1.mean()

0.3757706299334572

In [46]:
neural_network.forward(X_test)

array([[48.92120464],
       [10.57368911],
       [15.49928083],
       [ 9.52320448],
       [17.7202059 ],
       [22.14379837],
       [13.27506094],
       [21.46777833],
       [34.82995182],
       [24.55866132],
       [34.51984783],
       [19.31450329],
       [15.40330221],
       [42.24997428],
       [15.43757647],
       [24.6727321 ],
       [12.93763948],
       [32.52899067],
       [21.26766862],
       [21.5443723 ],
       [24.23761178],
       [25.85516356],
       [12.95008739],
       [23.70233444],
       [17.90598927],
       [25.59824611],
       [24.57648015],
       [37.8073911 ],
       [33.01267066],
       [21.5078249 ],
       [ 8.6292214 ],
       [ 7.99433943],
       [28.98184067],
       [21.43475598],
       [19.9866499 ],
       [45.87129637],
       [40.86205279],
       [23.40915193],
       [25.31201392],
       [27.36255706],
       [39.04851335],
       [21.12648538],
       [43.19596056],
       [17.20828741],
       [30.39511835],
       [27

In [189]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13,
                  activation=Sigmoid()),
            Dense(neurons=7,
                  activation=Sigmoid()),
            Dense(neurons=1,
                  activation=Linear())],
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(dl, SGD(lr=0.001))

trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 3000,
       eval_every = 300,
       seed=181008);

eval_regression_model(dl, X_test, y_test)

Validation loss after 300 epochs is 15.072
Validation loss after 600 epochs is 11.709
Validation loss after 900 epochs is 11.144
Validation loss after 1200 epochs is 10.025
Validation loss after 1500 epochs is 9.862
Validation loss after 1800 epochs is 9.462
Validation loss after 2100 epochs is 9.355
Loss increased after epoch 2400, final loss was 9.355, using the model from epoch 2100
Mean absolute error: 2.12

Mean squared error 9.46

Root mean squared error 3.08


#### Testing momentum

In [14]:
from lincoln.numpy.optimizers import SGDMomentum

In [38]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1)], 
            loss = MeanSquaredError(),
seed=190120)
trainer = Trainer(lr, SGD(lr=0.0001))

trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);
eval_regression_model(lr, X_test, y_test)

Validation loss after 10 epochs is 20838.548
Validation loss after 20 epochs is 7817.896
Validation loss after 30 epochs is 4833.318
Validation loss after 40 epochs is 4169.564
Validation loss after 50 epochs is 4032.698
Validation loss after 60 epochs is 4013.282
Validation loss after 70 epochs is 3998.216
Validation loss after 80 epochs is 3988.487
Validation loss after 90 epochs is 3978.392
Validation loss after 100 epochs is 3975.622


(3.5767, 26.1554)

#### "Neural Network" Regression

In [35]:
nn = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(nn, SGD(lr=0.0001))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 10,
       seed=10818,
       single_output=True);

Validation loss after 10 epochs is 6088.269
Validation loss after 20 epochs is 4895.965
Validation loss after 30 epochs is 4427.666
Validation loss after 40 epochs is 4128.529
Validation loss after 50 epochs is 3913.145
Validation loss after 60 epochs is 3739.412
Validation loss after 70 epochs is 3576.793
Validation loss after 80 epochs is 3432.877
Validation loss after 90 epochs is 3299.273
Validation loss after 100 epochs is 3186.684
Validation loss after 110 epochs is 3085.751
Validation loss after 120 epochs is 2993.504
Validation loss after 130 epochs is 2915.235
Validation loss after 140 epochs is 2839.337
Validation loss after 150 epochs is 2773.950
Validation loss after 160 epochs is 2717.566
Validation loss after 170 epochs is 2661.759
Validation loss after 180 epochs is 2607.947
Validation loss after 190 epochs is 2561.281
Validation loss after 200 epochs is 2518.434
Validation loss after 210 epochs is 2478.278
Validation loss after 220 epochs is 2443.188
Validation loss aft

In [36]:
eval_regression_model(nn, X_test, y_test)

(2.3526, 13.046)

Works!

#### "Deep Learning" Regression

#### Without momentum

In [41]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=7, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(dl, SGD(lr=0.0001))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 1000,
       eval_every = 1,
       seed=10818,
       single_output=True);

Validation loss after 1 epochs is 47901.615
Validation loss after 2 epochs is 29014.330
Validation loss after 3 epochs is 19685.898
Validation loss after 4 epochs is 15265.386
Validation loss after 5 epochs is 13196.414
Validation loss after 6 epochs is 12194.061
Validation loss after 7 epochs is 11574.913
Validation loss after 8 epochs is 11057.692
Validation loss after 9 epochs is 10555.229
Validation loss after 10 epochs is 10061.156
Validation loss after 11 epochs is 9554.197
Validation loss after 12 epochs is 9101.210
Validation loss after 13 epochs is 8647.923
Validation loss after 14 epochs is 8262.759
Validation loss after 15 epochs is 7934.661
Validation loss after 16 epochs is 7639.036
Validation loss after 17 epochs is 7383.042
Validation loss after 18 epochs is 7153.169
Validation loss after 19 epochs is 6928.515
Validation loss after 20 epochs is 6712.780
Validation loss after 21 epochs is 6519.403
Validation loss after 22 epochs is 6352.878
Validation loss after 23 epochs

In [42]:
eval_regression_model(dl, X_test, y_test)

(2.6381, 14.9998)

#### With momentum

In [47]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=7, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(dl, SGDMomentum(lr=0.001,
                                  momentum=0.1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

Validation loss after 10 epochs is 2897.001
Validation loss after 20 epochs is 1891.221
Validation loss after 30 epochs is 1711.341
Validation loss after 40 epochs is 1694.987
Validation loss after 50 epochs is 1420.211
Validation loss after 60 epochs is 1417.194
Validation loss after 70 epochs is 1356.096
Validation loss after 80 epochs is 1283.176
Loss increased after epoch 90, final loss was 1283.176, using model from epoch 80


In [48]:
eval_regression_model(dl, X_test, y_test)

(2.2003, 9.9315)

In [27]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer2 = Trainer(dl, SGDMomentum())
trainer2.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 10,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 4869.482
Validation loss after 20 epochs is 4867.908
Loss increased after epoch 30, final loss was 4867.908


In [28]:
eval_regression_model(dl, X_test, y_test)

(5.0027, 41.9342)

Works!

## Classification models

#### Data prep (SciKit Learn)

In [68]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [69]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

#### "`LogSigmoid`" Logistic Regression

In [70]:
# logr = NeuralNetwork(
#     layers=[Dense(neurons=1, 
#                   activation=LogSigmoid())],
#             loss = LogSigmoidLoss())

# trainer = Trainer(logr, SGD())
# trainer.fit(X_train, y_train, X_test, y_test,  
#        epochs = 100,
#        eval_every = 10,
#        seed=82618,
#        single_output=True);

In [71]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(logr_mse, SGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 1000,
       eval_every = 100,
       seed=82618,
       single_output=True);

setting up dense layer
Validation loss after 100 epochs is 11.251
Validation loss after 200 epochs is 5.576
Validation loss after 300 epochs is 4.307
Validation loss after 400 epochs is 4.096
Validation loss after 500 epochs is 3.988
Validation loss after 600 epochs is 3.908
Validation loss after 700 epochs is 3.844
Validation loss after 800 epochs is 3.791
Validation loss after 900 epochs is 3.749
Validation loss after 1000 epochs is 3.717


In [72]:
def accuracy(y_true, y_pred):
    return round(np.sum(np.equal(y_true, y_pred)) / y_true.shape[0], 4)

In [73]:
def eval_classification_model(model: NeuralNetwork, 
                              X_test: np.ndarray, 
                              y_test: np.ndarray,
                              log_probs: bool = False):
    preds = model.forward(X_test)
    if log_probs:
        preds = np.exp(preds)
    preds = preds > 0.5
    preds = preds.reshape(-1, 1)
    return accuracy(preds, y_test)

In [74]:
eval_classification_model(logr_mse, X_test, y_test)

0.9825

#### Testing softmax loss:

In [75]:
logr_soft = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(single_output=True),
seed=190120)

trainer = Trainer(logr_soft, SGD(0.01))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 1000,
       eval_every = 100,
       seed=82618,
       single_output=True);

setting up dense layer
Validation loss after 100 epochs is 120.267
Validation loss after 200 epochs is 117.588
Validation loss after 300 epochs is 117.355
Validation loss after 400 epochs is 117.257
Validation loss after 500 epochs is 117.188
Validation loss after 600 epochs is 117.128
Validation loss after 700 epochs is 117.076
Validation loss after 800 epochs is 117.024
Validation loss after 900 epochs is 116.975
Validation loss after 1000 epochs is 116.925


In [76]:
eval_classification_model(logr_soft, X_test, y_test)

0.9708

#### "Neural Network" Logistic Regression

In [77]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(single_output=True),
seed=190120)
trainer_m = Trainer(logr_nn, SGDMomentum())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 225.009
Validation loss after 20 epochs is 224.943
Validation loss after 30 epochs is 224.728
Validation loss after 40 epochs is 211.118
Validation loss after 50 epochs is 123.340
Validation loss after 60 epochs is 121.092
Validation loss after 70 epochs is 119.211
Validation loss after 80 epochs is 117.946
Validation loss after 90 epochs is 117.597
Validation loss after 100 epochs is 117.248


In [78]:
eval_classification_model(logr_nn, X_test, y_test)

0.9708

In [79]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(single_output=True),
seed=190120)
trainer_m = Trainer(logr_nn, SGD())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 1000,
       eval_every = 100,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 100 epochs is 225.019
Validation loss after 200 epochs is 224.888
Validation loss after 300 epochs is 224.316
Validation loss after 400 epochs is 125.520
Validation loss after 500 epochs is 122.149
Validation loss after 600 epochs is 120.017
Validation loss after 700 epochs is 118.549
Validation loss after 800 epochs is 117.641
Validation loss after 900 epochs is 117.144
Validation loss after 1000 epochs is 116.891


In [80]:
eval_classification_model(logr_nn, X_test, y_test)

0.9766

**Works!**

# Advanced Optimizers

## Adagrad

### Regression

In [81]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [82]:
from lincoln.numpy.optimizers import AdaGrad

In [89]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(lr, AdaGrad(1))

In [90]:
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 1,
       seed=10818,
       single_output=True);

setting up dense layer
Validation loss after 1 epochs is 46648.211
Validation loss after 2 epochs is 35773.610
Validation loss after 3 epochs is 28287.597
Validation loss after 4 epochs is 23505.051
Validation loss after 5 epochs is 19875.878
Validation loss after 6 epochs is 16886.621
Validation loss after 7 epochs is 14601.998
Validation loss after 8 epochs is 13035.690
Validation loss after 9 epochs is 11265.241
Validation loss after 10 epochs is 10067.982
Validation loss after 11 epochs is 9046.676
Validation loss after 12 epochs is 8215.858
Validation loss after 13 epochs is 7568.380
Validation loss after 14 epochs is 6929.361
Validation loss after 15 epochs is 6549.493
Validation loss after 16 epochs is 6063.891
Validation loss after 17 epochs is 5792.627
Validation loss after 18 epochs is 5477.881
Validation loss after 19 epochs is 5368.862
Validation loss after 20 epochs is 5058.642
Validation loss after 21 epochs is 4787.014
Validation loss after 22 epochs is 4734.230
Validati

In [91]:
eval_regression_model(lr, X_test, y_test)

(3.5662, 28.0392)

#### DL with AdaGrad

In [93]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=7, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(dl, AdaGrad(lr=0.5))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 200,
       eval_every = 20,
       seed=10818,
       single_output=True);

eval_regression_model(dl, X_test, y_test)

setting up dense layer
setting up dense layer
setting up dense layer
Validation loss after 20 epochs is 2694.116
Validation loss after 40 epochs is 2043.660
Validation loss after 60 epochs is 1678.514
Validation loss after 80 epochs is 1661.074
Loss increased after epoch 100, final loss was 1661.074


(2.1943, 11.3398)

### Classification

In [94]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [95]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [97]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(logr_mse, AdaGrad(lr=1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=82618,
       single_output=True);

setting up dense layer
Validation loss after 1 epochs is 10.506
Validation loss after 2 epochs is 9.180
Validation loss after 3 epochs is 5.476
Validation loss after 4 epochs is 4.089
Loss increased after epoch 5, final loss was 4.089


In [98]:
eval_classification_model(logr_nn, X_test, y_test)

0.9766

## Regularized SGD

### Regularization

In [99]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [100]:
from lincoln.numpy.optimizers import RegularizedSGD

In [101]:
lr = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(lr, RegularizedSGD(lr=0.001))

In [102]:
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=10818,
       single_output=True);

setting up dense layer
Validation loss after 1 epochs is 19857.889
Validation loss after 2 epochs is 7492.840
Validation loss after 3 epochs is 4617.919
Validation loss after 4 epochs is 4288.464
Validation loss after 5 epochs is 4192.022
Validation loss after 6 epochs is 4111.997
Validation loss after 7 epochs is 3984.513
Loss increased after epoch 8, final loss was 3984.513


In [103]:
eval_regression_model(lr, X_test, y_test)

(3.6796, 27.361)

### Deep Learning Regression

In [104]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=7, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(dl, RegularizedSGD(lr=0.01))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 1000,
       eval_every = 100,
       seed=10818,
       single_output=True);

eval_regression_model(dl, X_test, y_test)

setting up dense layer
setting up dense layer
setting up dense layer
Validation loss after 100 epochs is 23218.170
Validation loss after 200 epochs is 3185.410
Loss increased after epoch 300, final loss was 3185.410


(3.5932, 27.4456)

### Classification

In [105]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [106]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [109]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(logr_mse, RegularizedSGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 10,
       eval_every = 1,
       seed=82618,
       single_output=True);

setting up dense layer
Validation loss after 1 epochs is 49.231
Validation loss after 2 epochs is 33.163
Validation loss after 3 epochs is 27.212
Validation loss after 4 epochs is 23.281
Validation loss after 5 epochs is 21.082
Validation loss after 6 epochs is 19.604
Validation loss after 7 epochs is 18.482
Validation loss after 8 epochs is 17.563
Validation loss after 9 epochs is 16.820
Validation loss after 10 epochs is 16.165


In [110]:
eval_classification_model(logr_nn, X_test, y_test)

0.9766

In [111]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)
trainer_m = Trainer(logr_nn, RegularizedSGD())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 13.855
Validation loss after 20 epochs is 6.828
Validation loss after 30 epochs is 5.875
Validation loss after 40 epochs is 5.525
Validation loss after 50 epochs is 5.292
Validation loss after 60 epochs is 5.250
Validation loss after 70 epochs is 5.235
Loss increased after epoch 80, final loss was 5.235


In [112]:
eval_classification_model(logr_nn, X_test, y_test)

0.9708

In [114]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(single_output=True),
seed=190120)
trainer_m = Trainer(logr_nn, RegularizedSGD())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 225.159
Validation loss after 20 epochs is 224.700
Validation loss after 30 epochs is 223.292
Validation loss after 40 epochs is 133.085
Validation loss after 50 epochs is 125.478
Validation loss after 60 epochs is 123.491
Validation loss after 70 epochs is 122.454
Validation loss after 80 epochs is 121.967
Validation loss after 90 epochs is 121.663
Validation loss after 100 epochs is 121.479


In [115]:
eval_classification_model(logr_nn, X_test, y_test)

0.9708

### Dropout

In [116]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [117]:
dl = NeuralNetwork(
    layers=[Dense(neurons=13, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError())

trainer = Trainer(dl, AdaGrad(lr=1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 300,
       eval_every = 30,
       seed=10818,
       single_output=True);

eval_regression_model(dl, X_test, y_test)

setting up dense layer
setting up dense layer
Validation loss after 30 epochs is 2592.859
Validation loss after 60 epochs is 2536.203
Validation loss after 90 epochs is 2437.394
Validation loss after 120 epochs is 2420.111
Loss increased after epoch 150, final loss was 2420.111


(2.8601, 15.9505)

In [118]:
dl = NeuralNetwork(
    layers=[Dense(neurons=50, 
                  activation=Sigmoid(),
                  dropout=0.5),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError())

trainer = Trainer(dl, AdaGrad(lr=0.1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 10,
       seed=10818,
       single_output=True);

eval_regression_model(dl, X_test, y_test)

setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 9466.985
Validation loss after 20 epochs is 7928.233
Validation loss after 30 epochs is 7253.475
Validation loss after 40 epochs is 6750.645
Validation loss after 50 epochs is 6364.462
Validation loss after 60 epochs is 6126.840
Validation loss after 70 epochs is 5927.691
Validation loss after 80 epochs is 5830.308
Validation loss after 90 epochs is 5583.019
Validation loss after 100 epochs is 5473.189
Validation loss after 110 epochs is 5400.891
Validation loss after 120 epochs is 5298.735
Validation loss after 130 epochs is 5195.114
Validation loss after 140 epochs is 5099.220
Validation loss after 150 epochs is 5034.451
Validation loss after 160 epochs is 5007.961
Validation loss after 170 epochs is 4907.801
Validation loss after 180 epochs is 4809.196
Validation loss after 190 epochs is 4758.918
Validation loss after 200 epochs is 4737.912
Validation loss after 210 epochs is 4699.101
Validation loss af

(2.3836, 13.863)

In [119]:
dl = NeuralNetwork(
    layers=[Dense(neurons=50, 
                  activation=Sigmoid(),
                  dropout=0.5),
            Dense(neurons=1, 
                  activation=Linear())], 
            loss = MeanSquaredError())

trainer = Trainer(dl, AdaGrad(lr=0.1))
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 500,
       eval_every = 10,
       seed=10818,
       single_output=True);

eval_regression_model(dl, X_test, y_test)

setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 9466.985
Validation loss after 20 epochs is 7928.233
Validation loss after 30 epochs is 7253.475
Validation loss after 40 epochs is 6750.645
Validation loss after 50 epochs is 6364.462
Validation loss after 60 epochs is 6126.840
Validation loss after 70 epochs is 5927.691
Validation loss after 80 epochs is 5830.308
Validation loss after 90 epochs is 5583.019
Validation loss after 100 epochs is 5473.189
Validation loss after 110 epochs is 5400.891
Validation loss after 120 epochs is 5298.735
Validation loss after 130 epochs is 5195.114
Validation loss after 140 epochs is 5099.220
Validation loss after 150 epochs is 5034.451
Validation loss after 160 epochs is 5007.961
Validation loss after 170 epochs is 4907.801
Validation loss after 180 epochs is 4809.196
Validation loss after 190 epochs is 4758.918
Validation loss after 200 epochs is 4737.912
Validation loss after 210 epochs is 4699.101
Validation loss af

(2.3836, 13.863)

### Classification

In [120]:
from sklearn.datasets import load_breast_cancer
breast_cancer = load_breast_cancer()
data = breast_cancer.data
target = breast_cancer.target

In [121]:
from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=82618)

y_train, y_test = y_train.reshape(-1, 1), y_test.reshape(-1, 1)

In [129]:
logr_mse = NeuralNetwork(
    layers=[Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)

trainer = Trainer(logr_mse, RegularizedSGD())
trainer.fit(X_train, y_train, X_test, y_test,  
       epochs = 50,
       eval_every = 5,
       seed=82618,
       single_output=True);

setting up dense layer
Validation loss after 5 epochs is 21.082
Validation loss after 10 epochs is 16.165
Validation loss after 15 epochs is 13.738
Validation loss after 20 epochs is 11.835
Validation loss after 25 epochs is 10.070
Validation loss after 30 epochs is 8.251
Validation loss after 35 epochs is 6.155
Validation loss after 40 epochs is 5.348
Validation loss after 45 epochs is 5.003
Validation loss after 50 epochs is 4.885


In [123]:
eval_classification_model(logr_nn, X_test, y_test)

0.9708

In [130]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid()),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)
trainer_m = Trainer(logr_nn, RegularizedSGD())
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 100,
       eval_every = 10,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 10 epochs is 13.855
Validation loss after 20 epochs is 6.828
Validation loss after 30 epochs is 5.875
Validation loss after 40 epochs is 5.525
Validation loss after 50 epochs is 5.292
Validation loss after 60 epochs is 5.250
Validation loss after 70 epochs is 5.235
Loss increased after epoch 80, final loss was 5.235


In [125]:
eval_classification_model(logr_nn, X_test, y_test)

0.9766

In [131]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=30, 
                  activation=Sigmoid(),
                  dropout=0.8),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = MeanSquaredError(),
seed=190120)
trainer_m = Trainer(logr_nn, AdaGrad(0.1))
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 20,
       eval_every = 1,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 1 epochs is 10.057
Validation loss after 2 epochs is 7.439
Validation loss after 3 epochs is 6.774
Loss increased after epoch 4, final loss was 6.774


In [132]:
eval_classification_model(logr_nn, X_test, y_test)

0.9532

In [133]:
logr_nn = NeuralNetwork(
    layers=[Dense(neurons=200, 
                  activation=Sigmoid(),
                  dropout=0.8),
            Dense(neurons=1, 
                  activation=Sigmoid())],
            loss = LogSoftmaxLoss(single_output=True),
seed=190120)
trainer_m = Trainer(logr_nn, AdaGrad(0.1))
trainer_m.fit(X_train, y_train, X_test, y_test,  
       epochs = 20,
       eval_every = 1,
       seed=10818,
       single_output=True);

setting up dense layer
setting up dense layer
Validation loss after 1 epochs is 126.744
Validation loss after 2 epochs is 120.929
Validation loss after 3 epochs is 117.954
Validation loss after 4 epochs is 117.785
Loss increased after epoch 5, final loss was 117.785


In [134]:
eval_classification_model(logr_nn, X_test, y_test)

0.9649